In [1]:
import numpy as np
import pandas as pd

In [ ]:
"""데이터 출처 :https://www.kaggle.com/rsrishav/youtube-trending-video-dataset?select=KR_youtube_trending_data.csv
데이터 설명 : 유튜브 데일리 인기동영상 (한국)
dataurl : https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/youtube.csv"""
# Q1. 인기동영상 제작횟수가 많은 채널 상위 10개명을 출력하라 (날짜기준, 중복포함)

In [ ]:
# Q2. 논란으로 인기동영상이 된 케이스를 확인하고 싶다. dislikes수가 like 수보다 높은 동영상을 제작한 채널을 모두 출력하라

In [ ]:
# Q3. 채널명을 바꾼 케이스가 있는지 확인하고 싶다.
# channelId의 경우 고유값이므로 이를 통해 채널명을 한번이라도 바꾼 채널의 갯수를 구하여라

In [ ]:
# Q4. 일요일에 인기있었던 영상들중 가장많은 영상 종류(categoryId)는 무엇인가?

In [ ]:
# Q5. 각 요일별 인기 영상들의 categoryId는 각각 몇개 씩인지 하나의 데이터 프레임으로 표현하라

In [ ]:
# Q6. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.
# viewcount대비 댓글수가 가장 높은 영상을 확인하라 (view_count값이 0인 경우는 제외한다)

In [ ]:
# Q7. 댓글의 수로 (comment_count) 영상 반응에 대한 판단을 할 수 있다.
# viewcount대비 댓글수가 가장 낮은 영상을 확인하라 (view_counts, ratio값이 0인경우는 제외한다.)

In [ ]:
# Q8. like 대비 dislike의 수가 가장 적은 영상은 무엇인가? (like, dislike 값이 0인경우는 제외한다)

In [ ]:
# Q9. 가장많은 트렌드 영상을 제작한 채널의 이름은 무엇인가? (날짜기준, 중복포함)

In [ ]:
# Q10. 20회(20일)이상 인기동영상 리스트에 포함된 동영상의 숫자는?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/kukuroo3/youtube-episodic-contents-kr(참고, 데이터 수정)
데이터 설명 : 유튜브 “공범” 컨텐츠 동영상 정보 ( 10분 간격 수집)
dataurl1 (비디오 정보) = https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/videoInfo.csv
dataurl2 (참가자 채널 정보)= https://raw.githubusercontent.com/Datamanim/datarepo/main/youtube/channelInfo.csv"""
# Q11. 각 데이터의 ‘ct’컬럼을 시간으로 인식할수 있게 datatype을 변경하고
# video 데이터의 videoname의 각 value 마다 몇개의 데이터씩 가지고 있는지 확인하라

In [ ]:
# Q12. 수집된 각 video의 가장 최신화 된 날짜의 viewcount값을 출력하라

In [ ]:
# Q13. Channel 데이터중 2021-10-03일 이후 각 채널의 처음 기록 됐던 구독자 수(subcnt)를 출력하라

In [ ]:
# Q14. 각채널의 2021-10-03 03:00:00 ~ 2021-11-01 15:00:00 까지 구독자수 (subcnt) 의 증가량을 구하여라

In [ ]:
# Q15. 각 비디오는 10분 간격으로 구독자수, 좋아요, 싫어요수, 댓글수가 수집된것으로 알려졌다.
# 공범 EP1의 비디오정보 데이터중 수집간격이 5분 이하, 20분이상인 데이터 구간( 해당 시점 전,후) 의 시각을 모두 출력하라

In [ ]:
# Q16. 각 에피소드의 시작날짜(년-월-일)를 에피소드 이름과 묶어 데이터 프레임으로 만들고 출력하라

In [ ]:
# Q17. “공범” 컨텐츠의 경우 19:00시에 공개 되는것으로 알려져있다.
# 공개된 날의 21시의 viewcnt, ct, videoname 으로 구성된 데이터 프레임을 viewcnt를 내림차순으로 정렬하여 출력하라

In [ ]:
# Q18. video 정보의 가장 최근 데이터들에서 각 에피소드의 싫어요/좋아요 비율을 ratio 컬럼으로 만들고
# videoname, ratio로 구성된 데이터 프레임을 ratio를 오름차순으로 정렬하라

In [ ]:
# Q19. 2021-11-01 00:00:00 ~ 15:00:00까지 각 에피소드별 viewcnt의 증가량을 데이터 프레임으로 만드시오

In [ ]:
# Q20. video 데이터 중에서 중복되는 데이터가 존재한다. 중복되는 각 데이터의 시간대와 videoname 을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/darinhawley/fifa-world-cup-goalscorers-19302018(참고, 데이터 수정)
데이터 설명 : 1930 ~2018년도 월드컵 출전선수 골기록
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/worldcup/worldcupgoals.csv"""
# Q21. 주어진 전체 기간의 각 나라별 골득점수 상위 5개 국가와 그 득점수를 데이터프레임형태로 출력하라

In [ ]:
# Q22. 주어진 전체기간동안 골득점을 한 선수가 가장 많은 나라 상위 5개 국가와 그 선수 숫자를 데이터 프레임 형식으로 출력하라

In [ ]:
# Q23. Years 컬럼은 년도 -년도 형식으로 구성되어있고, 각 년도는 4자리 숫자이다.
# 년도 표기가 4자리 숫자로 안된 케이스가 존재한다. 해당 건은 몇건인지 출력하라

In [ ]:
# Q24. Q3에서 발생한 예외 케이스를 제외한 데이터프레임을 df2라고 정의하고 데이터의 행의 숫자를 출력하라
# (아래 문제부터는 df2로 풀이하겠습니다)

In [ ]:
# Q25. 월드컵 출전횟수를 나타내는 ‘LenCup’ 컬럼을 추가하고 4회 출전한 선수의 숫자를 구하여라

In [ ]:
# Q26. Yugoslavia 국가의 월드컵 출전횟수가 2회인 선수들의 숫자를 구하여라

In [ ]:
# Q27. 2002년도에 출전한 전체 선수는 몇명인가?

In [ ]:
# Q28. 이름에 ‘carlos’ 단어가 들어가는 선수의 숫자는 몇 명인가? (대, 소문자 구분 x)

In [ ]:
# Q29. 월드컵 출전 횟수가 1회뿐인 선수들 중에서 가장 많은 득점을 올렸던 선수는 누구인가?

In [ ]:
# Q30. 월드컵 출전횟수가 1회 뿐인 선수들이 가장 많은 국가는 어디인가?

In [ ]:
"""데이터 출처 :https://www.data.go.kr/data/15051872/fileData.do(참고, 데이터 수정)
데이터 설명 : 서울특별시_공공자전거 시간대별 이용정보
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/bicycle/seoul_bi.csv"""
# Q31. 대여일자별 데이터의 수를 데이터프레임으로 출력하고, 가장 많은 데이터가 있는 날짜를 출력하라

In [ ]:
# Q32. 각 일자의 요일을 표기하고 (‘Monday’ ~’Sunday’) ‘day_name’컬럼을 추가하고 이를 이용하여
# 각 요일별 이용 횟수의 총합을 데이터 프레임으로 출력하라

In [ ]:
# Q33. 각 요일별 가장 많이 이용한 대여소의 이용횟수와 대여소 번호를 데이터 프레임으로 출력하라

In [ ]:
# Q34. 나이대별 대여구분 코드의 (일일권/전체횟수) 비율을 구한 후 가장 높은 비율을 가지는 나이대를 확인하라.
# 일일권의 경우 일일권 과 일일권(비회원)을 모두 포함하라

In [ ]:
# Q35. 연령대별 평균 이동거리를 구하여라

In [ ]:
# Q36. 연령대 코드가 20대인 데이터를 추출하고,이동거리값이 추출한 데이터의 이동거리값의 평균 이상인 데이터를 추출한다.
# 최종 추출된 데이터를 대여일자, 대여소 번호 순서로 내림차순 정렬 후 1행부터 200행까지의 탄소량의 평균을 소숫점 3째 자리까지 구하여라

In [ ]:
# Q37. 6월 7일 ~10대의 “이용건수”의 중앙값은?

In [ ]:
# Q38. 평일 (월~금) 출근 시간대(오전 6,7,8시)의 대여소별 이용 횟수를 구해서 데이터 프레임 형태로 표현한 후
# 각 대여시간별 이용 횟수의 상위 3개 대여소와 이용횟수를 출력하라

In [ ]:
# Q39. 이동거리의 평균 이상의 이동거리 값을 가지는 데이터를 추출하여 추출데이터의 이동거리의 표본표준편차 값을 구하여라

In [ ]:
# Q40. 남성(‘M’ or ‘m’)과 여성(‘F’ or ‘f’)의 이동거리값의 평균값을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/unsdsn/world-happiness(참고, 데이터 수정)
데이터 설명 : 전세계 행복도 지표 조사
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/happy2/happiness.csv"""
# Q41. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다.
# 각년도의 행복랭킹 10위를 차지한 나라의 행복점수의 평균을 구하여라

In [ ]:
# Q42. 데이터는 2018년도와 2019년도의 전세계 행복 지수를 표현한다.
# 각년도의 행복랭킹 50위이내의 나라들의 각각의 행복점수 평균을 데이터프레임으로 표시하라

In [ ]:
# Q43. 2018년도 데이터들만 추출하여 행복점수와 부패에 대한 인식에 대한 상관계수를 구하여라

In [ ]:
# Q44. 2018년도와 2019년도의 행복랭킹이 변화하지 않은 나라명의 수를 구하여라

In [ ]:
# Q45. 2019년도 데이터들만 추출하여 각변수간 상관계수를 구하고 내림차순으로 정렬한 후
# 상위 5개를 데이터 프레임으로 출력하라. 컬럼명은 v1,v2,corr으로 표시하라

In [ ]:
# Q46. 각 년도별 하위 행복점수의 하위 5개 국가의 평균 행복점수를 구하여라

In [ ]:
# Q47. 2019년 데이터를 추출하고 해당데이터의 상대 GDP 평균 이상의 나라들과
# 평균 이하의 나라들의 행복점수 평균을 각각 구하고 그 차이값을 출력하라

In [ ]:
# Q48. 각년도의 부패에 대한인식을 내림차순 정렬했을때 상위 20개 국가의 부패에 대한인식의 평균을 구하여라

In [ ]:
# Q49. 2018년도 행복랭킹 50위 이내에 포함됐다가 2019년 50위 밖으로 밀려난 국가의 숫자를 구하여라

In [ ]:
# Q50. 2018년,2019년 모두 기록이 있는 나라들 중 년도별 행복점수가 가장 증가한 나라와 그 증가 수치는?

In [ ]:
"""데이터 출처 : https://archive.ics.uci.edu/ml/datasets/Power+consumption+of+Tetouan+city (참고, 데이터 수정)
데이터 설명 : 기온, 습도,바람풍속에 따른 도시의 3개 지역구의 에너지 소비량
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/consum/Tetuan City power consumption.csv"""
# Q51. DateTime컬럼을 통해 각 월별로 몇개의 데이터가 있는지 데이터 프레임으로 구하여라

In [ ]:
# Q52. 3월달의 각 시간대별 온도의 평균들 중 가장 낮은 시간대의 온도를 출력하라

In [ ]:
# Q53. 3월달의 각 시간대별 온도의 평균들 중 가장 높은 시간대의 온도를 출력하라

In [ ]:
# Q54. Zone 1 Power Consumption 컬럼의 value값의 크기가
# Zone 2 Power Consumption 컬럼의 value값의 크기보다 큰 데이터들의 Humidity의 평균을 구하여라

In [ ]:
# Q55. 각 zone의 에너지 소비량의 상관관계를 구해서 데이터 프레임으로 표기하라

In [ ]:
# Q56. Temperature의 값이 10미만의 경우 A, 10이상 20미만의 경우 B,20이상 30미만의 경우 C,
# 그 외의 경우 D라고 할때 각 단계의 데이터 숫자를 구하여라

In [ ]:
# Q57. 6월 데이터중 12시의 Temperature의 표준편차를 구하여라

In [ ]:
# Q58. 6월 데이터중 12시의 Temperature의 분산을 구하여라

In [ ]:
# Q59. Temperature의 평균이상의 Temperature의 값을 가지는 데이터를 Temperature를 기준으로 정렬 했을때
# 4번째 행의 Humidity 값은?

In [ ]:
# Q60. Temperature의 중간값 이상의 Temperature의 값을 가지는 데이터를Temperature를 기준으로 정렬 했을때
# 4번째 행의 Humidity 값은?

In [ ]:
"""데이터 출처 : https://www.kaggle.com/abcsds/pokemon (참고, 데이터 수정)
데이터 설명 : 포켓몬 정보
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/pok/Pokemon.csv"""
# Q61. Legendary 컬럼은 전설포켓몬 유무를 나타낸다.전설포켓몬과 그렇지 않은 포켓몬들의 HP평균의 차이를 구하여라

In [ ]:
# Q62. Type 1은 주속성 Type 2 는 부속성을 나타낸다. 가장 많은 부속성 종류는 무엇인가?

In [ ]:
# Q63. 가장 많은 Type 1 의 종의 평균 Attack 을 평균 Defense로 나눈값은?

In [ ]:
# Q64. 포켓몬 세대(Generation) 중 가장많은 Legendary를 보유한 세대는 몇세대인가?

In [ ]:
# Q65. ‘HP’, ‘Attack’, ‘Defense’, ‘Sp. Atk’, ‘Sp. Def’, ‘Speed’ 간의 상관 계수중
# 가장 절댓값이 큰 두 변수와 그 값을 구하여라

In [ ]:
# Q66. 각 Generation의 Attack으로 오름차순 정렬시 상위 3개 데이터들(18개)의 Attack의 전체 평균을 구하여라

In [ ]:
# Q67. 각 Generation의 Attack으로 내림차순 정렬시 상위 5개 데이터들(30개)의 Attack의 전체 평균을 구하여라

In [ ]:
# Q68. 가장 흔하게 발견되는 (Type1 , Type2) 의 쌍은 무엇인가?

In [ ]:
# Q69. 한번씩만 존재하는 (Type1 , Type2)의 쌍의 갯수는 몇개인가?

In [ ]:
# Q70. 한번씩만 존재하는 (Type1 , Type2)의 쌍을 각 세대(Generation)은 각각 몇개씩 가지고 있는가?

In [ ]:
"""데이터 출처 : 국민체육진흥공단 (문화 빅데이터플랫폼) (참고, 데이터 수정)
데이터 설명 : 대한민국 국민 체력장 평가
data url = https://raw.githubusercontent.com/Datamanim/datarepo/main/body/body.csv"""
# Q71. 전체데이터의 수축기혈압(최고) - 이완기혈압(최저)의 평균을 구하여라

In [ ]:
# Q72. 50~59세의 신장평균을 구하여라

In [ ]:
# Q73. 연령대 (20~29 : 20대 …) 별 인원수를 구하여라

In [ ]:
# Q74. 연령대 (20~29 : 20대 …) 별 등급의 숫자를 데이터 프레임으로 표현하라

In [ ]:
# Q75. 남성 중 A등급과 D등급의 체지방률 평균의 차이(큰 값에서 작은 값의 차)를 구하여라

In [ ]:
# Q76. 여성 중 A등급과 D등급의 체중의 평균의 차이(큰 값에서 작은 값의 차)를 구하여라

In [ ]:
# Q77. bmi는 자신의 몸무게(kg)를 키의 제곱(m)으로 나눈값이다.
# 데이터의 bmi 를 구한 새로운 컬럼을 만들고 남성의 bmi 평균을 구하여라

In [ ]:
# Q78. bmi보다 체지방율이 높은 사람들의 체중평균을 구하여라

In [ ]:
# Q79. 남성과 여성의 악력 평균의 차이를 구하여라

In [ ]:
# Q80. 남성과 여성의 교차윗몸일으키기 횟수의 평균의 차이를 구하여라

In [ ]:
"""데이터 출처 : https://data.kma.go.kr/cmmn/static/staticPage.do?page=intro
데이터 설명 : 2020년도 이화동(서울) , 수영동(부산)의 시간단위의 기온과 강수량
데이터 url : https://raw.githubusercontent.com/Datamanim/datarepo/main/weather/weather2.csv"""
# Q81. 여름철(6월,7월,8월) 이화동이 수영동보다 높은 기온을 가진 시간대는 몇개인가?

In [ ]:
# Q82. 이화동과 수영동의 최대강수량의 시간대를 각각 구하여라

In [ ]:
"""데이터 출처 : https://www.kaggle.com/shubh0799/churn-modelling 에서 변형
데이터 설명 : 고객의 신상정보 데이터를 통한 회사 서비스 이탈 예측 (종속변수 : Exited)
Data url : https://raw.githubusercontent.com/Datamanim/datarepo/main/churn/train.csv"""
# Q83. 남성 이탈(Exited)이 가장 많은 국가(Geography)는 어디이고 이탈 인원은 몇명인가?

In [ ]:
# Q84. 카드를 소유(HasCrCard ==1)하고 있으면서 활성멤버(IsActiveMember ==1) 인 고객들의
# 평균 나이를 소숫점이하 4자리까지 구하여라?

In [ ]:
# Q85. Balance 값이 중간값 이상을 가지는 고객들의 CreditScore의 표준편차를 소숫점이하 3자리까지 구하여라

In [ ]:
"""데이터 출처 :https://www.data.go.kr/data/15007122/fileData.do (참고, 데이터 수정)
데이터 설명 : 2018년도 성인의 건강검 진데이터 (흡연상태 1- 흡연, 0-비흡연 )
데이터 url : https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/train.csv"""
# Q86. 수축기혈압과 이완기 혈압기 수치의 차이를 새로운 컬럼(‘혈압차’) 으로 생성하고,
#  연령대 코드별 각 그룹 중 ‘혈압차’ 의 분산이 5번째로 큰 연령대 코드를 구하여라

In [ ]:
# Q87. 비만도를 나타내는 지표인 WHtR는 허리둘레 / 키로 표현한다. 일반적으로 0.58이상이면 비만으로 분류한다.
#  데이터중 WHtR 지표상 비만인 인원의 남/여 비율을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/anmolkumar/health-insurance-cross-sell-prediction(참고, 데이터 수정)
데이터 설명 : 자동차 보험 가입 예측
Data url : https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/train.csv"""
# Q88. Vehicle_Age 값이 2년 이상인 사람들만 필터링 하고 그중에서
#  Annual_Premium 값이 전체 데이터의 중간값 이상인 사람들을 찾고, 그들의 Vintage값의 평균을 구하여라

In [ ]:
# Q89. vehicle_age에 따른 각 성별(gender)그룹의 Annual_Premium값의 평균을 구하여 아래 테이블과 동일하게 구현하라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/iabhishekofficial/mobile-price-classification?select=train.csv(참고, 데이터 수정)
데이터 설명 : 핸드폰 가격예측 (price_range컬럼 0(저렴) ~3(매우비쌈) 범위 )
Data url https://raw.githubusercontent.com/Datamanim/datarepo/main/mobile/train.csv"""
# Q90. price_range 의 각 value를 그룹핑하여 각 그룹의 n_cores 의 빈도가 가장높은 value와 그 빈도수를 구하여라

In [ ]:
# Q91. price_range 값이 3인 그룹에서 상관관계가 2번째로 높은 두 컬럼과 그 상관계수를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/teejmahal20/airline-passenger-satisfaction?select=train.csv (참고, 데이터 수정)
데이터 설명 : 비행탑승 경험 만족도 (satisfaction 컬럼 : ‘neutral or dissatisfied’ or satisfied ) (83123, 24) shape
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/train.csv"""
# Q92. Arrival Delay in Minutes 컬럼이 결측치인 데이터들 중 ‘neutral or dissatisfied’ 보다
# ‘satisfied’의 수가 더 높은 Class는 어디 인가?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/adityakadiwal/water-potability
데이터 설명 : 수질 음용성 여부 (Potablillity 컬럼 : 0 ,1 )
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/train.csv"""
# Q93. ph값은 상당히 많은 결측치를 포함한다. 결측치를 제외한 나머지 데이터들 중 사분위값 기준 하위 25%의 값들의 평균값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mirichoi0218/insurance/code(참고, 데이터 수정)
데이터 설명 : 의료비용 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/MedicalCost/train.csv"""
# Q94. 흡연자와 비흡연자 각각 charges의 상위 10% 그룹의 평균의 차이는?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/harlfoxem/housesalesprediction (참고, 데이터 수정)
데이터 설명 : 킹카운티 주거지 가격 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/kingcountyprice/train.csv"""
# Q95. bedrooms 의 빈도가 가장 높은 값을 가지는 데이터들의 price의 상위 10%와 하위 10%값의 차이를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mohansacharya/graduate-admissions(참고, 데이터 수정)
데이터 설명 : 대학원 입학 가능성 예측
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/admission/train.csv"""
# Q96. Serial No. 컬럼을 제외하고 ‘Chance of Admit’을 종속변수, 나머지 변수를 독립변수라 할때,
#  랜덤포레스트를 통해 회귀 예측을 할 떄 변수중요도 값을 출력하라 (시드값에 따라 순서는 달라질수 있음)

In [ ]:
"""데이터 출처 :https://www.kaggle.com/uciml/red-wine-quality-cortez-et-al-2009(참고, 데이터 수정)
데이터 설명 : 레드 와인 퀄리티 예측문제
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/redwine/train.csv"""
# Q97. quality 값이 3인 그룹과 8인 데이터그룹의 각 컬럼별 독립변수의 표준편차 값의 차이를 구할때
# 그값이 가장 큰 컬럼명을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/prathamtripathi/drug-classification(참고, 데이터 수정)
데이터 설명 : 투약하는 약을 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/drug/train.csv"""
# Q98. 남성들의 연령대별 (10살씩 구분 0~9세 10~19세 …) Na_to_K값의 평균값을 구해서 데이터 프레임으로 표현하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/sid321axn/audit-data(참고, 데이터 수정)
데이터 설명 : 사기회사 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/audit/train.csv"""
# Q99. 데이터의 Risk 값에 따른 score_a와 score_b의 평균값을 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/kyr7plus/emg-4(참고, 데이터 수정)
데이터 설명 : 센서데이터로 동작 유형 분류
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/muscle/train.csv"""
# Q100. pose값에 따른 각 motion컬럼의 중간값의 가장 큰 차이를 보이는 motion컬럼은 어디이며 그값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/mysarahmadbhat/hyundai-used-car-listing(참고, 데이터 수정)
데이터 설명 : 현대 차량가격 분류문제
Data url :hhttps://raw.githubusercontent.com/Datamanim/datarepo/main/hyundai/train.csv"""
# Q101. 정보(row수)가 가장 많은 상위 3차종의 price값의 각 평균값은?

In [ ]:
"""데이터 출처 :https://www.kaggle.com/pritsheta/diabetes-dataset(참고, 데이터 수정)
데이터 설명 : 당뇨여부 판단하기
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/diabetes/train.csv"""
# Q102. Outcome 값에 따른 각 그룹의 각 컬럼의 평균 차이를 구하여라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/pritsheta/netflix-stock-data-from-2002-to-2021(참고, 데이터 수정)
데이터 설명 : 넷플릭스 주식데이터
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/nflx/NFLX.csv"""
# Q103. 매년 5월달의 open가격의 평균값을 데이터 프레임으로 표현하라

In [ ]:
"""데이터 출처 :https://www.kaggle.com/datasets/vivovinco/nba-player-stats(참고, 데이터 수정)
Data url :https://raw.githubusercontent.com/Datamanim/datarepo/main/nba/nba.csv"""
# Q104. Tm 컬럼은 각 팀의 이름을 의미한다. TOR팀의 평균나이를 소수 4째 자리까지 구하여라

In [ ]:
# Q105. Pos 컬럼은 포지션을 의미한다. 전체 선수 중 최소나이대의 선수들을 필터하고 그들 중 가장 많은 포지션은 무엇인지 확인하라

In [ ]:
# Q106. 선수들의 이름은 first_name+ 공백 + last_name으로 이루어져 있다.
# 가장 많은 first_name은 무엇이며 몇 회 발생하는지 확인하라

In [ ]:
# Q107. PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라

In [ ]:
# Q108. PTS컬럼은 경기당 평균득점수 이다. 각포지션별로 경기당 평균득점수의 평균을 구하여라

In [ ]:
# Q109. G컬럼은 참여한 경기의 숫자이다. 각 팀별로 가장 높은 경기참여 수를 가진 선수들의 경기 참여 숫자의 평균을 구하여라

In [ ]:
# Q110. Tm의 값이 MIA이며 Pos는 C또는 PF인 선수의 MP값의 평균은?

In [ ]:
# Q111. 전체 데이터중 G의 평균값의 1.5배 이상인 데이터들만 추출했을때 3P값의 평균은?

In [ ]:
# Q112. Age의 평균 이상인 그룹과 평균 미만인 그룹간의 G값의 평균의 차이는?

In [ ]:
# Q113. 평균나이가 가장 젊은 팀은 어디인가

In [ ]:
# Q114. Pos그룹별 평균 MP값을 구하여라